In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# SAM 3 インタラクティブ画像セグメンテーション (SAM 3 Interactive Image Segmentation)

このノートブックでは、Jupyter Widgetを使用して、SAM 3 モデルによるインタラクティブな画像セグメンテーションを体験できます。
画像をアップロードまたはURLから読み込み、テキストプロンプトやボックスプロンプトを使ってオブジェクトをセグメント化できます。

# <a target="_blank" href="https://colab.research.google.com/github/facebookresearch/sam3/blob/main/notebooks/sam3_image_interactive.ipynb">
#   <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
# </a>

## Google Colab セットアップ

Google Colabで実行する場合は、以下のセルを実行してください。

In [ ]:
# Google Colabで実行しているかチェック
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except:
    IN_COLAB = False
    print("ローカル環境で実行中")

In [ ]:
# Google Colabの場合、必要なパッケージをインストール
if IN_COLAB:
    !pip install -q pycocotools
    # SAM3のインストール (GitHubリポジトリから)
    !pip install -q git+https://github.com/facebookresearch/sam3.git
    print("パッケージのインストールが完了しました")

In [ ]:
# Google Driveをマウント（データをドライブに保存している場合）
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Driveがマウントされました")
    print("データのパスは /content/drive/MyDrive/... のように指定してください")

### データのアップロード方法

Google Colabでデータを使用する方法は3つあります:

1. **Google Driveを使用**: 上のセルでマウント後、`/content/drive/MyDrive/` 以下のパスを指定
2. **ファイルを直接アップロード**: 左側のファイルアイコンから手動アップロード
3. **Colabのファイルアップロードウィジェットを使用**: 以下のコードを実行

```python
from google.colab import files
uploaded = files.upload()  # ファイル選択ダイアログが表示されます
```

## 環境設定 (Environment Set-up)

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib scikit-learn
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam3.git'

In [ ]:
%matplotlib widget

In [ ]:
import torch
# Ampere GPU向けにtfloat32を有効化
# https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# ノートブック全体でbfloat16を使用。カードがサポートしていない場合はfloat16を試してください
if torch.cuda.is_available():
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

# ノートブック全体で推論モードを使用。勾配が必要な場合は無効にしてください
torch.inference_mode().__enter__()

## モデルの読み込み (Load the model)

In [ ]:
import sam3
from sam3 import build_sam3_image_model
import os
sam3_root = os.path.join(os.path.dirname(sam3.__file__), "..")
bpe_path = f"{sam3_root}/assets/bpe_simple_vocab_16e6.txt.gz"

In [ ]:
model = build_sam3_image_model(bpe_path=bpe_path)

In [ ]:
from sam3.model.sam3_image_processor import Sam3Processor
processor = Sam3Processor(model)

## Jupyter ウィジェット (Jupyter widget)

以下のコードは、インタラクティブなセグメンテーションツールを構築するためのクラス定義です。
画像のアップロード、URL入力、テキストプロンプト入力、ボックス描画などのUIコンポーネントを作成し、イベントハンドラを設定しています。

In [ ]:
import io

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import requests
from IPython.display import clear_output, display, HTML
from matplotlib.patches import Rectangle


class Sam3SegmentationWidget:
    """テキストとボックスプロンプトを使用したSAM3セグメンテーションのためのインタラクティブなJupyterウィジェット。"""

    def __init__(self, processor):
        """
        セグメンテーションウィジェットを初期化します。

        Args:
            processor: Sam3Processor インスタンス
        """
        self.processor = processor
        self.state = None
        self.current_image = None
        self.current_image_array = None
        self.box_mode = "positive"
        self.drawing_box = False
        self.box_start = None
        self.current_rect = None

        self._setup_ui()
        self._setup_plot()

    def _setup_ui(self):
        """UIコンポーネントをセットアップします。"""
        self.upload_widget = widgets.FileUpload(
            accept="image/*", multiple=False, description="画像をアップロード"
        )
        self.upload_widget.observe(self._on_image_upload, names="value")

        self.url_input = widgets.Text(
            placeholder="または画像のURLを入力",
        )
        self.url_button = widgets.Button(description="URLを読み込む", button_style="info")
        self.url_button.on_click(self._on_load_url)
        url_box = widgets.HBox(
            [self.url_input, self.url_button],
            layout=widgets.Layout(width="100%", justify_content="space-between"),
        )

        self.text_input = widgets.Text(
            placeholder='セグメンテーションプロンプトを入力 (例: "person", "dog")',
            continuous_update=False,
        )
        self.text_input.observe(self._on_text_submit, names="value")
        self.text_button = widgets.Button(description="セグメント実行", button_style="success")
        self.text_button.on_click(self._on_text_prompt)
        text_box = widgets.HBox(
            [self.text_input, self.text_button],
            layout=widgets.Layout(width="100%", justify_content="space-between"),
        )

        self.box_mode_buttons = widgets.ToggleButtons(
            options=["Positive Boxes", "Negative Boxes"],
            description="ボックスモード:",
            button_style="",
            tooltips=[
                "含めるオブジェクトの周りにボックスを描画",
                "除外するオブジェクトの周りにボックスを描画",
            ],
        )
        self.box_mode_buttons.observe(self._on_box_mode_change, names="value")

        self.clear_button = widgets.Button(
            description="すべてのプロンプトをクリア", button_style="warning"
        )
        self.clear_button.on_click(self._on_clear_prompts)

        self.confidence_slider = widgets.FloatSlider(
            value=0.5,
            min=0.0,
            max=1.0,
            step=0.01,
            description="信頼度:",
            continuous_update=False,
            style={"description_width": "initial"},
        )
        self.confidence_slider.observe(self._on_confidence_change, names="value")

        self.size_slider = widgets.IntSlider(
            value=960,
            min=300,
            max=2000,
            step=10,
            description="画像サイズ:",
            continuous_update=False,
            style={"description_width": "initial"},
        )
        self.size_slider.observe(self._on_size_change, names="value")

        slider_box = widgets.HBox(
            [self.confidence_slider, self.size_slider],
            layout=widgets.Layout(justify_content="space-between"),
        )

        self.output = widgets.Output()
        self.status_label = widgets.Label(value="開始するには画像をアップロードしてください")

        # このボックスはmatplotlibの出力を保持し、CSSでターゲットにできます。
        self.plot_container = widgets.Box([self.output])
        self.plot_container.add_class("no-drag")

        # matplotlibキャンバス上のカーソルを十字にするCSS
        css_style = widgets.HTML(
            """
        <style>
            .jupyter-matplotlib-canvas, canvas {
                cursor: crosshair !important;
            }
        </style>
        """
        )
        # 各アコーディオンペイン用のVBoxを作成
        source_pane = widgets.VBox([self.upload_widget, url_box])
        prompt_pane = widgets.VBox(
            [
                widgets.Label("テキストプロンプト:"),
                text_box,
                self.box_mode_buttons,
                self.confidence_slider,
                self.clear_button,
            ]
        )
        display_pane = widgets.VBox([self.size_slider])

        # コントロールペインを保持するアコーディオンを作成
        self.accordion = widgets.Accordion(
            children=[source_pane, prompt_pane, display_pane]
        )
        self.accordion.set_title(0, "画像ソース")
        self.accordion.set_title(1, "セグメンテーションプロンプト")
        self.accordion.set_title(2, "表示設定")
        self.accordion.selected_index = 0  # 最初のペインを開いた状態で開始

        # コントロール用の左サイドバーを作成
        sidebar = widgets.VBox(
            [self.status_label, widgets.HTML("<h4>コントロール</h4>"), self.accordion]
        )
        sidebar.layout = widgets.Layout(
            width="380px",
            min_width="380px",
            max_width="380px",
            border="1px solid #e0e0e0",
            padding="10px",
            margin="0 15px 0 0",
            flex="0 0 auto",
        )

        # 画像表示用のメインエリアを作成
        main_area = widgets.VBox([self.plot_container])
        main_area.layout = widgets.Layout(flex="1", min_width="500px", overflow="auto")

        # サイドバーとメインエリアを最終的なアプリレイアウトに結合
        app_layout = widgets.HBox([sidebar, main_area])
        app_layout.layout = widgets.Layout(
            width="100%",
            display="flex",
            flex_flow="row",
            align_items="stretch",
        )

        # メインコンテナを設定
        self.container = widgets.VBox(
            [
                css_style,
                widgets.HTML("<h3>🖼️ SAM3 インタラクティブセグメンテーション</h3>"),
                app_layout,
            ]
        )

    def _setup_plot(self):
        """matplotlibの図をセットアップします。"""
        # plt.ioff()
        self.fig, self.ax = plt.subplots(figsize=(12, 8))
        # plt.ion()
        self.ax.axis("off")
        self.fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
        self.fig.canvas.toolbar_visible = False
        self.fig.canvas.header_visible = False
        self.fig.canvas.footer_visible = False
        self.fig.canvas.resizable = False

        # plt.close(self.fig)

    def _set_loading(self, is_loading, message="処理中..."):
        """読み込み状態を表示/非表示にし、コントロールを無効/有効にします。"""
        if is_loading:
            self.status_label.value = f"⏳ {message}"
            self.upload_widget.disabled = True
            self.url_button.disabled = True
            self.text_button.disabled = True
            self.clear_button.disabled = True
            self.box_mode_buttons.disabled = True
            self.confidence_slider.disabled = True
        else:
            self.upload_widget.disabled = False
            self.url_button.disabled = False
            self.text_button.disabled = False
            self.clear_button.disabled = False
            self.box_mode_buttons.disabled = False
            self.confidence_slider.disabled = False

    def _on_image_upload(self, change):
        """画像アップロードを処理します。"""
        if change["new"]:
            uploaded_file = change["new"][0]
            image = PIL.Image.open(io.BytesIO(uploaded_file["content"])).convert("RGB")
            self._set_image(image)

    def _on_load_url(self, button):
        """URLからの画像読み込みを処理します。"""
        url = self.url_input.value.strip()
        if not url:
            self.status_label.value = "URLを入力してください"
            return

        self._set_loading(True, "URLから画像をダウンロード中...")

        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            image = PIL.Image.open(io.BytesIO(response.content)).convert("RGB")
            self._set_image(image)
        except Exception as e:
            self._set_loading(False)
            self.status_label.value = f"画像の読み込みエラー: {str(e)}"

    def _set_image(self, image):
        """現在の画像を設定し、図のサイズを調整し、状態を初期化します。"""
        self._set_loading(True, "モデルで画像を処理中...")

        try:

            self.current_image = image
            self.current_image_array = np.array(image)
            self.state = self.processor.set_image(image)
            self._set_loading(False)
            self.status_label.value = (
                f"画像を読み込みました: {image.size[0]}x{image.size[1]} ピクセル"
            )
            self._resize_figure()
            self._update_display()
            self._connect_plot_events()
            self.accordion.selected_index = 1
        except Exception as e:
            self._set_loading(False)
            self.status_label.value = f"画像処理エラー: {str(e)}"

    def _on_text_submit(self, change):
        """Enterキーによるテキストプロンプト送信を処理します。"""
        # ボタンクリックと同じハンドラを呼び出す
        self._on_text_prompt(None)

    def _on_text_prompt(self, button):
        """テキストプロンプト送信を処理します。"""
        if self.state is None:
            self.status_label.value = "まず画像を読み込んでください"
            return

        prompt = self.text_input.value.strip()
        if not prompt:
            self.status_label.value = "プロンプトを入力してください"
            return

        self._set_loading(True, f'プロンプト "{prompt}" でセグメント中...')

        try:
            self.state = self.processor.set_text_prompt(prompt, self.state)
            self._set_loading(False)
            self.status_label.value = f'プロンプト "{prompt}" でセグメント完了'
            self._update_display()
        except Exception as e:
            self._set_loading(False)
            self.status_label.value = f"エラー: {str(e)}"

    def _on_box_mode_change(self, change):
        """ボックスモードの切り替えを処理します。"""
        self.box_mode = "positive" if change["new"] == "Positive Boxes" else "negative"

    def _on_clear_prompts(self, button):
        """すべてのプロンプトをクリアし、画像のみの状態にリセットします。"""
        if self.current_image is not None:
            try:
                self._set_loading(True, "プロンプトをクリアしてリセット中...")
                self.state = self.processor.reset_all_prompts(self.state)
                if "prompted_boxes" in self.state:
                    del self.state["prompted_boxes"]
                self.text_input.value = ""
                self._set_loading(False)
                self.status_label.value = "すべてのプロンプトをクリアしました"
                self._update_display()
            except Exception as e:
                self._set_loading(False)
                import traceback

                self.status_label.value = f"エラー: {str(e)} {traceback.format_exc()}"

    def _on_confidence_change(self, change):
        """信頼度閾値の変更を処理します。"""
        if self.state is not None:
            self.state = self.processor.set_confidence_threshold(
                change["new"], self.state
            )
            self._update_display()

    def _connect_plot_events(self):
        """ボックス描画用のmatplotlibイベントハンドラを接続します。"""
        # カスタムボックス描画を可能にするためにmatplotlibのツールバーナビゲーションを無効にする
        if hasattr(self.fig.canvas, "toolbar") and self.fig.canvas.toolbar is not None:
            self.fig.canvas.toolbar.pan()
            self.fig.canvas.toolbar.pan()

        self.fig.canvas.mpl_connect("button_press_event", self._on_press)
        self.fig.canvas.mpl_connect("button_release_event", self._on_release)
        self.fig.canvas.mpl_connect("motion_notify_event", self._on_motion)

    def _on_press(self, event):
        """ボックス描画のマウスプレスを処理します。"""
        if event.inaxes != self.ax:
            return
        self.drawing_box = True
        self.box_start = (event.xdata, event.ydata)

    def _on_motion(self, event):
        """ボックスプレビューのマウス移動を処理します。"""
        if not self.drawing_box or event.inaxes != self.ax or self.box_start is None:
            return

        if self.current_rect is not None:
            self.current_rect.remove()

        x0, y0 = self.box_start
        x1, y1 = event.xdata, event.ydata
        width = x1 - x0
        height = y1 - y0

        color = "green" if self.box_mode == "positive" else "red"
        self.current_rect = Rectangle(
            (x0, y0),
            width,
            height,
            fill=False,
            edgecolor=color,
            linewidth=2,
            linestyle="--",
        )
        self.ax.add_patch(self.current_rect)
        self.fig.canvas.draw_idle()

    def _on_release(self, event):
        """ボックスを確定するためのマウスリリースを処理します。"""
        if not self.drawing_box or event.inaxes != self.ax or self.box_start is None:
            self.drawing_box = False
            return

        self.drawing_box = False

        if self.current_rect is not None:
            self.current_rect.remove()
            self.current_rect = None

        if self.state is None:
            return

        x0, y0 = self.box_start
        x1, y1 = event.xdata, event.ydata

        x_min = min(x0, x1)
        x_max = max(x0, x1)
        y_min = min(y0, y1)
        y_max = max(y0, y1)

        if abs(x_max - x_min) < 5 or abs(y_max - y_min) < 5:
            return

        # 画像の寸法を取得
        img_h = self.state["original_height"]
        img_w = self.state["original_width"]

        # xyxyピクセル座標からcxcywh正規化形式に変換
        center_x = (x_min + x_max) / 2.0 / img_w
        center_y = (y_min + y_max) / 2.0 / img_h
        width = (x_max - x_min) / img_w
        height = (y_max - y_min) / img_h

        box = [center_x, center_y, width, height]
        label = self.box_mode == "positive"
        mode_str = "positive" if label else "negative"

        # 表示用にプロンプトボックスをピクセル座標で保存
        if "prompted_boxes" not in self.state:
            self.state["prompted_boxes"] = []
        self.state["prompted_boxes"].append(
            {"box": [x_min, y_min, x_max, y_max], "label": label}
        )
        
        if mode_str == "positive":
            loading_msg = "ポジティブボックスを追加して再セグメント中..."
        else:
            loading_msg = "ネガティブボックスを追加して再セグメント中..."

        self._set_loading(True, loading_msg)

        try:
            self.state = self.processor.add_geometric_prompt(box, label, self.state)
            self._set_loading(False)
            if mode_str == "positive":
                self.status_label.value = "ポジティブボックスを追加しました"
            else:
                self.status_label.value = "ネガティブボックスを追加しました"
            self._update_display()
        except Exception as e:
            self._set_loading(False)
            self.status_label.value = f"ボックス追加エラー: {str(e)}"

    def _resize_figure(self):
        """画像とスライダーの値に基づいて新しい図のサイズを計算して適用します。"""
        if self.current_image is None:
            return

        # 1. 元の画像の寸法を取得
        img_w, img_h = self.current_image.size

        # 2. スライダーの値が表示の直接のターゲット幅になります
        display_w = float(self.size_slider.value)

        # 3. 元のアスペクト比を維持するために対応する高さを計算します
        aspect_ratio = img_h / img_w
        display_h = int(display_w * aspect_ratio)

        # 4. ピクセル寸法をMatplotlib用のインチに変換して適用します
        dpi = self.fig.dpi
        new_figsize = (display_w / dpi, display_h / dpi)
        self.fig.set_size_inches(new_figsize, forward=True)

    def _on_size_change(self, change):
        """画像サイズスライダーからの変更を処理します。"""
        if self.current_image is not None:
            self._resize_figure()
            # キャンバスのサイズを変更した後、コンテンツを再描画する必要があります
            self._update_display()

    def _update_display(self):
        """現在の結果で表示を更新します。"""
        if self.current_image_array is None:
            return

        with self.output:
            clear_output(wait=True)

            self.ax.clear()
            self.ax.axis("off")
            self.ax.imshow(self.current_image_array)

            if self.state is not None and "masks" in self.state:
                masks = self.state.get("masks", [])
                boxes = self.state.get("boxes", [])
                scores = self.state.get("scores", [])

                if len(masks) > 0:
                    mask_overlay = np.zeros((*self.current_image_array.shape[:2], 4))

                    for i, (mask, box, score) in enumerate(zip(masks, boxes, scores)):
                        mask_np = mask[0].cpu().numpy()

                        color = plt.cm.tab10(i % 10)[:3]
                        mask_overlay[mask_np > 0.5] = (*color, 0.5)

                        x0, y0, x1, y1 = box.cpu().numpy()
                        rect = Rectangle(
                            (x0, y0),
                            x1 - x0,
                            y1 - y0,
                            fill=False,
                            edgecolor=color,
                            linewidth=2,
                        )
                        self.ax.add_patch(rect)

                        self.ax.text(
                            x0,
                            y0 - 5,
                            f"{score:.2f}",
                            color="white",
                            fontsize=10,
                            bbox=dict(
                                facecolor=color, alpha=0.7, edgecolor="none", pad=2
                            ),
                        )

                    self.ax.imshow(mask_overlay)
                    self.status_label.value = f"{len(masks)} 個のオブジェクトが見つかりました"
                else:
                    self.status_label.value = (
                        "信頼度閾値を超えるオブジェクトは見つかりませんでした"
                    )

            # プロンプトボックスを破線で表示
            if self.state is not None and "prompted_boxes" in self.state:
                for prompted_box in self.state["prompted_boxes"]:
                    box_coords = prompted_box["box"]
                    is_positive = prompted_box["label"]

                    x0, y0, x1, y1 = box_coords
                    color = "green" if is_positive else "red"

                    rect = Rectangle(
                        (x0, y0),
                        x1 - x0,
                        y1 - y0,
                        fill=False,
                        edgecolor=color,
                        linewidth=2,
                        linestyle="--",
                    )
                    self.ax.add_patch(rect)

            # display(self.fig.canvas)

    def display(self):
        display(self.container)

    # ノートブックでの表示をより便利にするためにこれを追加
    def _ipython_display_(self):
        self.display()


# 実行! (Run!)

In [ ]:
widget = Sam3SegmentationWidget(processor)
widget.display()